In [1]:
%config IPCompleter.greedy=True
%matplotlib inline

In [2]:
import tensorflow as tf
import pandas as pd, numpy as np, os, sys
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from IPython.display import display, HTML
from IPython.core.debugger import Tracer
from IPython.core.debugger import Pdb

font = {'size'   : 18}
matplotlib.rc('font', **font)

def plotHorizontalBars(series, xlabel, title):
    xpos = np.arange(len(series.index), 0, -1)
    plt.barh(xpos, series, align='center', alpha=0.5, color='g')
    plt.grid(axis='x')
    plt.yticks(xpos, series.index)
    plt.xlabel(xlabel)
    plt.title(title)

def plotVerticalBars(series, ylabel, title):
    xpos = np.arange(len(series.index), 0, -1)
    plt.bar(xpos, series, align='center', alpha=0.5, color='g')
    plt.grid(axis='y')
    plt.xticks(xpos, series.index)
    plt.ylabel(ylabel)
    plt.title(title)

/home/hwang7/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

In [4]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [5]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


In [6]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


In [7]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


In [8]:
x1 = tf.Variable(1)
print(x1.graph is tf.get_default_graph())

True


In [9]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())
tf.reset_default_graph()

True
False


In [10]:
w = tf.constant(5)
x = w+2
y=x+5
z=x*3
with tf.Session() as sess:
    yval, zval = sess.run([y,z])
    print([yval, zval])

[12, 21]


In [11]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

m,n=housing.data.shape
print('prepend a 1-column:')
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
print(housing_data_plus_bias)

prepend a 1-column:
[[   1.            8.3252       41.         ...,    2.55555556   37.88
  -122.23      ]
 [   1.            8.3014       21.         ...,    2.10984183   37.86
  -122.22      ]
 [   1.            7.2574       52.         ...,    2.80225989   37.85
  -122.24      ]
 ..., 
 [   1.            1.7          17.         ...,    2.3256351    39.43
  -121.22      ]
 [   1.            1.8672       18.         ...,    2.12320917   39.43
  -121.32      ]
 [   1.            2.3886       16.         ...,    2.61698113   39.37
  -121.24      ]]


In [12]:
X = tf.constant(housing_data_plus_bias, dtype = tf.float32, name ="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name ="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y) 
with tf.Session() as sess: 
    theta_value = theta.eval()
    print(theta_value)

[[ -3.74651413e+01]
 [  4.35734153e-01]
 [  9.33829229e-03]
 [ -1.06622010e-01]
 [  6.44106984e-01]
 [ -4.25131839e-06]
 [ -3.77322501e-03]
 [ -4.26648885e-01]
 [ -4.40514028e-01]]


In [ ]:
n_epochs = 1000
learning_rate = 0.01

m,n=housing_data_plus_bias.shape
x = tf.constant(housing_data_plus_bias, dtype = tf.float32, name ="x")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name ="y")

# theta are the coefficients
theta = tf.Variable(tf.random_uniform([n, 1], -1.0, 1.0))
y_pred = tf.matmul(x, theta)
mse = tf.reduce_sum(tf.square(y_pred - y))
#gradients = 2/m * tf.matmul(tf.transpose(X), error)
#gradients = tf.gradients(mse, [theta])[0]
#training_op = tf.assign(theta, theta - learning_rate * gradients)
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print('x:')
    print(x.eval())
    print('y:')
    print(y.eval())
    print('y_pred:')
    print(y_pred.eval())
    print('theta:')
    print(theta.eval())
    for epoch in range(n_epochs):
        print('epoch =', epoch)
        print('Epoch', epoch, 'MSE =', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
    print('best theta:', best_theta)

In [20]:
logDir = '/home/hwang7/log'
os.makedirs(logDir, exist_ok=True)

with tf.name_scope('variables') as scope:
    W = tf.Variable([.3], dtype=tf.float32)
    b = tf.Variable([-.3], dtype=tf.float32)

with tf.name_scope('inputs') as scope:
    x = tf.placeholder(tf.float32)
    y = tf.placeholder(tf.float32)
with tf.name_scope('loss') as scope:
    linear_model = W*x + b
    squared_deltas = tf.square(linear_model - y)
    loss = tf.reduce_sum(squared_deltas)
with tf.name_scope('optimize') as scope:
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = optimizer.minimize(loss)

with tf.name_scope('graph_output') as scope:
    from datetime import datetime
    now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
    runLogDir = '{}/run-{}/'.format(logDir, now)

    loss_summary = tf.summary.scalar('LOSS', loss)
    file_writer = tf.summary.FileWriter(runLogDir, tf.get_default_graph())

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init) # reset values to incorrect defaults.
    for i in range(1000):
        if i % 200 == 0:
            summary_str = loss_summary.eval({x: [1, 2, 3, 4], y: [0, -1, -2, -3]})
            file_writer.add_summary(summary_str, i/200)
        sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})
    print(sess.run([W, b]))

file_writer.close()

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


In [21]:
print(linear_model.op.name)
print(squared_deltas.op.name)
print(loss.op.name)

loss_2/add
loss_2/Square
loss_2/Sum


In [67]:
tf.reset_default_graph()
def relu(x):
    w_shape = (int(x.get_shape()[1]), 1)
    print(w_shape)
    w = tf.Variable(tf.random_normal(w_shape), name='weights')
    b = tf.Variable(0.0, name='bias')
    z = tf.add(tf.matmul(x, w), b, name='z')
    return tf.maximum(z, 0.0, name='relu')

n_features=3
x = tf.placeholder(tf.float32, shape=(None, n_features), name='x')
relus = [relu(x) for i in range(5)]
output = tf.add_n(relus, name='output')

init = tf.global_variables_initializer()
xs = {x:[[1,2,3]]}
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(relus, xs))
    print(output.eval(xs))

(3, 1)
(3, 1)
(3, 1)
(3, 1)
(3, 1)
[array([[ 4.60321331]], dtype=float32), array([[ 3.90952158]], dtype=float32), array([[ 0.7526083]], dtype=float32), array([[ 4.59001446]], dtype=float32), array([[ 5.51728725]], dtype=float32)]
[[ 19.37264633]]
